# IRIS Bench Vs Redis

The aim of this notebook is to compare the performance between IRIS In-Memory database and Redis

In [1]:
# import the iris module to make the connection to the database
import iris
# import redis
import redis
# import random to generate random numbers
import random
# import tqdm to show progress bars
import tqdm
# import colorama to color the output
import colorama

it will make 100 000 writes and then 100 000 reads

In [2]:
numWrites = 100000
numReads  = 100000

## Create helper functions

In [3]:
## Getter and Setter
# set a global variable
def setter(connection, key, value):
    """
    Set a key to a value in the database
    """
    connection[key] = value

# get a global variable
def getter(connection, key):
    """
    Get a key from the database
    """
    return connection[key]


## Commun
# Writer
def write(setter,connection,numKeys):
    """
    Write numKeys keys to the database
    """
    
    # create a global reference to the database
    
    # create a progress bar
    iter = tqdm.tqdm(range(numKeys), "Writing", bar_format='{l_bar}{bar}')
    
    # iterate over the progress bar
    for i in iter:
        # write a key and a value to the database
        setter(connection, 'key'+str(i), 'val'+str(random.randint(0,999999)))
    
    # return the elapsed time
    return(iter.format_dict["elapsed"])

# Reader
def random_read(getter,connection,numKeys, numReads):
    """
    Read numReads random keys from the database
    """

    # create a progress bar
    iter = tqdm.tqdm(range(numReads), "Reading", bar_format='{l_bar}{bar}')

    # iterate over the progress bar
    for i in iter:
        # read a random key from the database
        getter(connection, 'key'+str(random.randint(1,numKeys)-1))
    
    # return the elapsed time
    return(iter.format_dict["elapsed"])

# Print result
def print_results(product, action, time, count):
    """
    Print the results of the benchmark
    """

    print(f"\n{colorama.Fore.CYAN}{product}{colorama.Style.RESET_ALL} {action}:{colorama.Fore.YELLOW}{count:,d}{colorama.Style.RESET_ALL}/{colorama.Fore.YELLOW}{time:5.2f}{colorama.Style.RESET_ALL} ⇒ {colorama.Fore.GREEN}{count/time:12,.2f}{colorama.Style.RESET_ALL} msg/sec\n\n")


## Redis

In [4]:
# make the connection to the database
redisConnection = redis.Redis()

print_results("Redis", "Writes", write(setter,redisConnection,numWrites), numWrites)
print_results("Redis", "Reads ", random_read(getter,redisConnection,numWrites, numReads), numReads)


Writing: 100%|██████████



Redis Writes:100,000/15.45 ⇒     6,473.03 msg/sec




Reading: 100%|██████████


Redis Reads :100,000/16.08 ⇒     6,218.63 msg/sec




## IRIS

In [5]:
# make the connection to the database
irisConnection = iris.gref('^exampleKV')

print_results("IRIS", "Writes", write(setter,irisConnection,numWrites), numWrites)
print_results("IRIS", "Reads ", random_read(getter,irisConnection,numWrites, numReads), numReads)

Writing: 100%|██████████



IRIS Writes:100,000/ 0.28 ⇒   358,105.54 msg/sec




Reading: 100%|██████████


IRIS Reads :100,000/ 0.32 ⇒   310,206.18 msg/sec


